# Documentação de Requisitos do Projeto

## Objetivo

Criar um projeto que, primeiramente atenda aos requisitos de solução do case proposto pela empresa LOFT, para a vaga Analista de planejamento Senior e posteriormente ampliar as possibilidades.

## Contexto

Você recebeu três bases de dados relacionadas a clientes, contratos e acionamentos de cobrança de uma empresa. Seu objetivo é cruzar essas bases para realizar uma análise que ajude a entender o perfil dos clientes, o comportamento dos contratos e a eficácia dos canais de cobrança.
Bases:
1. Clientes: informações demográficas e perfil dos clientes (idade, cidade, renda mensal).
2. Contratos: contratos vinculados aos clientes, com valor, data de início e status de pagamento (em dia, atrasado, inadimplente).
3. Acionamentos: registros dos contatos feitos para cobrança, com tipo de contato (WhatsApp, telefone, email) e data.

## Tarefas sugeridas:
- análise exploratória
- sugestões e propostas estratégicas
- Identificar perfil dos clientes inadimplentes
- Analisar a distribuição dos status de pagamento
- Avaliar quais canais de cobrança são mais usados e sua relação com a regularização dos pagamentos
- Sugerir ações para melhorar a recuperação de crédito
- Explorar outras análises que julgar relevantes para o negócio

## Entregáveis:
- Relatório com insights e visualizações
- Código ou planilha com as análises

## Restrições
- Nenhuma declarada
<br><br>
*Desenvolvido por: Cleucio Diogo Bastos Ferreira*

# Preparação do ambiente

In [70]:
!pip install xlsxwriter
# !pip install pyxlsb
!pip install polars --upgrade
!pip install fastexcel
# !pip install gspread_formatting

## Importação de bibliotecas

In [71]:
import pandas as pd
import polars as pl
# import time, glob
import numpy as np
import os, gspread, unicodedata, re
import seaborn as sns
from scipy import stats
# from tqdm import tqdm
# from pyxlsb import open_workbook
from openpyxl import load_workbook
from datetime import date, datetime,timedelta
from dateutil import parser
from typing import Dict, Union, List
from pathlib import Path
import matplotlib.pyplot as plt

# #abrindo conexão com o drive
from google.colab import drive

## Definição de Métodos personalizados

In [72]:
"""
Biblioteca de Funções Auxiliares para Análise de Dados
Compatível com Pandas e Polars
"""

# Uso: conteudo = testar_acesso('/caminho/para/pasta')
def testar_acesso(caminho_da_pasta: str) -> list:
  """
  Verifica se um caminho existe e retorna seu conteúdo.

  Args:
    caminho_da_pasta: Caminho completo da pasta

  Returns:
    Lista com nomes de arquivos/pastas ou lista vazia se não existir
  """
  print(f"Verificando acesso à pasta: '{caminho_da_pasta}'...")

  if os.path.exists(caminho_da_pasta):
    print("✅ Acesso bem-sucedido!")
    return os.listdir(caminho_da_pasta)
  else:
    print(f"❌ ERRO: O caminho não foi encontrado.")
    return []

# Uso: gerar_outputs_excel({'Vendas': df1, 'Estoque': df2}, 'relatorio.xlsx', './outputs')
def gerar_outputs_excel(
  dataframes_dict: Dict[str, Union[pd.DataFrame, pl.DataFrame]],
  file_name: str,
  output_path: str
):
  """
  Salva múltiplos DataFrames em um arquivo Excel (uma planilha por DataFrame).

  Args:
    dataframes_dict: Dicionário {nome_planilha: dataframe}
    file_name: Nome do arquivo Excel (ex: 'relatorio.xlsx')
    output_path: Caminho da pasta de saída
  """
  os.makedirs(output_path, exist_ok=True)
  caminho_completo = os.path.join(output_path, file_name)

  with pd.ExcelWriter(caminho_completo, engine='xlsxwriter') as writer:
    for sheet_name, df in dataframes_dict.items():
      if isinstance(df, pl.DataFrame):
        df = df.to_pandas()

      df.to_excel(writer, sheet_name=sheet_name, index=False)
      writer.sheets[sheet_name].autofit()

  print(f"✅ Excel gerado em: {caminho_completo}")


# ==================== LIMPEZA E TRANSFORMAÇÃO ====================

# Uso: df = limpar_colunas(df, ['preco', 'valor'])
def limpar_colunas(df: pd.DataFrame, colunas: List[str]) -> pd.DataFrame:
  """
  Limpa e padroniza colunas de texto (remove quebras, espaços, formata números).

  Args:
    df: DataFrame Pandas
    colunas: Lista de colunas para limpar

  Returns:
    DataFrame modificado
  """
  for col in colunas:
    df[col] = (
      df[col]
      .apply(str)
      .str.lower()
      .str.replace('\n', '', regex=False)
      .str.replace('r$', '', regex=False)
      .str.replace('.', '', regex=False)
      .str.replace(',', '.', regex=False)
      .str.replace('nan', '')
      .str.strip()
    )
  return df

# Uso: df = to_numeric(df, ['preco', 'quantidade'])
def to_numeric(df: pd.DataFrame, colunas: List[str]) -> pd.DataFrame:
  """
  Converte colunas para tipo numérico (trata formato brasileiro).

  Args:
    df: DataFrame Pandas
    colunas: Lista de colunas para converter

  Returns:
    DataFrame com colunas numéricas
  """
  for col in colunas:
    if pd.api.types.is_string_dtype(df[col]):
      df[col] = (
        df[col]
        .replace({r'^\s*$': np.nan}, regex=True)
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
      )
    df[col] = pd.to_numeric(df[col], errors='coerce')
  return df

# Uso: df = brazilian_format(df, ['preco', 'total'])
def brazilian_format(df: pd.DataFrame, colunas: List[str]) -> pd.DataFrame:
  """
  Formata números para padrão brasileiro (1.234,56).

  Args:
    df: DataFrame Pandas
    colunas: Lista de colunas numéricas para formatar

  Returns:
    DataFrame com valores formatados
  """
  for coluna in colunas:
    df[coluna] = df[coluna].apply(
      lambda x: f"{float(x):,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
      if pd.notnull(x) and isinstance(x, (int, float)) else x
    )
  return df

# Uso: valor_float = money_to_float('1.234,56')
def money_to_float(valor: str) -> float:
  """
  Converte string em formato monetário brasileiro para float.

  Args:
    valor: String ou número (ex: '1.234,56' ou 1234.56)

  Returns:
    Float com o valor convertido
  """
  if isinstance(valor, (int, float)):
    return float(valor)
  if valor == '':
    return 0.0
  return float(valor.replace('.', '').replace(',', '.'))

# Uso: valor_str = float_to_money(1234.56)
def float_to_money(valor: float) -> str:
  """
  Converte float para formato monetário brasileiro.

  Args:
    valor: Número a ser formatado

  Returns:
    String formatada (ex: '1.234,56')
  """
  if isinstance(valor, (int, float)):
    return f'{valor:,.2f}'.replace(',', '').replace('.', ',')
  return ''

# ==================== NORMALIZAÇÃO ====================

# Uso: df = normalize_columns(df, case='lower')
def normalize_columns(
  df: Union[pd.DataFrame, pl.DataFrame],
  case: str = 'lower'
) -> Union[pd.DataFrame, pl.DataFrame]:
  """
  Normaliza nomes das colunas (remove acentos, espaços viram underscores).

  Args:
    df: DataFrame Pandas ou Polars
    case: 'lower' ou 'upper' (padrão: 'lower')

  Returns:
    DataFrame com colunas normalizadas
  """
  original_columns = df.columns
  new_columns = []

  for col in original_columns:
    # Remove acentos
    col_sem_acentos = "".join(
      c for c in unicodedata.normalize('NFD', col)
      if unicodedata.category(c) != 'Mn'
    )

    # Converte case
    col_cased = col_sem_acentos.lower() if case == 'lower' else col_sem_acentos.upper()

    # Limpa espaços
    col_final = col_cased.strip().replace(' ', '_')
    new_columns.append(col_final)

  rename_map = dict(zip(original_columns, new_columns))

  if isinstance(df, pl.DataFrame):
    return df.rename(rename_map)
  elif isinstance(df, pd.DataFrame):
    return df.rename(columns=rename_map)
  else:
    raise TypeError(f"Tipo não suportado: {type(df)}")

# Uso: df = normalize_string_data(df, columns=['nome', 'cidade'], case='lower')
def normalize_string_data(
  df: pl.DataFrame,
  columns: List[str] = None,
  case: str = 'lower',
  replace_space_with: str = '_'
) -> pl.DataFrame:
  """
  Normaliza dados de texto em colunas (remove acentos, padroniza case).

  Args:
    df: DataFrame Polars
    columns: Lista de colunas (None = todas as strings)
    case: 'lower' ou 'upper' (padrão: 'lower')
    replace_space_with: Caractere para substituir espaços (padrão: '_')

  Returns:
    DataFrame com dados normalizados
  """
  # Determina colunas alvo
  if columns is None:
    target_columns = [col for col in df.columns if df[col].dtype == pl.String]
  else:
    target_columns = [col for col in columns if col in df.columns and df[col].dtype == pl.String]

  if not target_columns:
    print("Nenhuma coluna de string válida. Retornando DataFrame original.")
    return df

  def remove_accents(text: str) -> str:
    if text is None:
      return None
    return "".join(
      c for c in unicodedata.normalize('NFD', text)
      if unicodedata.category(c) != 'Mn'
    )

  expressions = []
  for col_name in target_columns:
    expr = pl.col(col_name)
    expr = expr.map_elements(remove_accents, return_dtype=pl.String)
    expr = expr.str.strip_chars().str.replace_all(r"\s+", replace_space_with)
    expr = expr.str.strip_chars().str.replace_all(" ", replace_space_with)
    expr = expr.str.to_lowercase() if case == 'lower' else expr.str.to_uppercase()
    expr = expr.str.replace('\n', '')
    expr = expr.str.replace_all(r"[^a-zA-Z0-9-_+/*#@!%&|]", "")
    expressions.append(expr.alias(col_name))

  return df.with_columns(*expressions)

# ==================== ANÁLISE E VALIDAÇÃO ====================

# Uso: verificar_nulos(df, min_pct_nulos=5.0, show_all=False)
def verificar_nulos(df: pl.DataFrame, min_pct_nulos: float = 0.0, show_all: bool = False):
  """
  Analisa e exibe porcentagem de valores nulos por coluna.

  Args:
    df: DataFrame Polars
    min_pct_nulos: Porcentagem mínima para exibir (0-100)
    show_all: Se True, mostra todas as colunas com nulos
  """
  print("\n--- Análise de Valores Nulos ---")

  null_counts_df = df.null_count()
  total_rows = df.shape[0]

  if total_rows == 0:
    print("DataFrame vazio.")
    return

  found_nulls = False
  for col in df.columns:
    null_count = null_counts_df[col][0]
    if null_count > 0:
      found_nulls = True
      pct = (null_count / total_rows) * 100
      if show_all or (pct >= min_pct_nulos):
        print(f"  • {col}: {null_count} nulos ({pct:.2f}%)")

  if not found_nulls:
    print("Nenhum valor nulo encontrado.")

# Uso: df_limpo = filter_null_and_empty(df, ['nome', 'cpf'])
def filter_null_and_empty(df: pl.DataFrame, columns_to_check: List[str]) -> pl.DataFrame:
  """
  Remove linhas com valores nulos ou vazios nas colunas especificadas.

  Args:
    df: DataFrame Polars
    columns_to_check: Lista de colunas para verificar

  Returns:
    DataFrame filtrado
  """
  for col_name in columns_to_check:
    if col_name not in df.columns:
      raise ValueError(f"Coluna '{col_name}' não encontrada.")

  condicoes = []
  for c in columns_to_check:
    if df[c].dtype == pl.String:
      condicao = pl.col(c).is_not_null() & (pl.col(c).str.strip_chars() != "")
    else:
      condicao = pl.col(c).is_not_null()
    condicoes.append(condicao)

  if not condicoes:
    return df

  return df.filter(pl.all_horizontal(*condicoes))

# ==================== VISUALIZAÇÃO ====================

# Uso: print_dataframe(df, type='head', n_rows=10)
def print_dataframe(
  dataframe: Union[pd.DataFrame, pl.DataFrame],
  type: str = 'full',
  width: int = 300,
  n_rows: int = -1
):
  """
  Exibe DataFrame sem truncar (suporta Pandas e Polars).

  Args:
    dataframe: DataFrame a ser exibido
    type: 'full', 'head' ou 'tail'
    width: Largura máxima em caracteres
    n_rows: Número de linhas (-1 = todas)
  """
  if isinstance(dataframe, pl.DataFrame):
    with pl.Config(tbl_rows=n_rows, tbl_cols=-1, tbl_width_chars=width):
      if type == 'full':
        print(dataframe)
      elif type == 'head':
        print(dataframe.head(20 if n_rows < 0 else n_rows))
      else:
        print(dataframe.tail(20 if n_rows < 0 else n_rows))

  elif isinstance(dataframe, pd.DataFrame):
    original_max_rows = pd.get_option('display.max_rows')
    original_max_cols = pd.get_option('display.max_columns')
    original_width = pd.get_option('display.width')

    try:
      pd.set_option('display.max_rows', n_rows)
      pd.set_option('display.max_columns', None)
      pd.set_option('display.width', width)

      if type == 'full':
        display(dataframe)
      elif type == 'head':
        display(dataframe.head())
      else:
        display(dataframe.tail())
    finally:
      pd.set_option('display.max_rows', original_max_rows)
      pd.set_option('display.max_columns', original_max_cols)
      pd.set_option('display.width', original_width)
  else:
    raise TypeError(f"Tipo não suportado: {type(dataframe)}")

########################################

def print_dataframe_schema(df:pl.DataFrame):
  """
  Recebe um DataFrame Polars e imprime o schema (Coluna | Tipo)
  em formato tabular.
  """
  # Cria um novo DF apenas com os metadados
  df_schema = pl.DataFrame({
    "column_name": df.schema.keys(),
    "dtype": [str(t) for t in df.schema.values()] # Converte o objeto Tipo para String
  })

  print_dataframe(df_schema)
  # return df_schema

## Testando acesso ao drive

In [73]:
raiz = '/content/drive/'
drive.mount(raiz)

file_path = r'/content/drive/MyDrive/00. Cases/case_loft_202512/'
input_path = file_path+'input/'
output_path = file_path+'output/'

#testando se o acesso a pasta esta ok
lista_de_arquivos = testar_acesso(file_path)

# Verifique o que foi retornado e imprima o resultado
if lista_de_arquivos:
  print('\nArquivos e pastas encontrados:')
  for item in lista_de_arquivos:
    print(f'  - {item}')
else:
  print('\nNenhum arquivo ou pasta foi encontrado no local especificado.')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Verificando acesso à pasta: '/content/drive/MyDrive/00. Cases/case_loft_202512/'...
✅ Acesso bem-sucedido!

Arquivos e pastas encontrados:
  - input
  - output
  - case_202512.pbix
  - Leia me.pdf
  - case_loft_20251209.ipynb


# Importando os dados

In [74]:
target = input_path + '/case_2512.xlsx'

# chave = nome do DF; valor = nome da aba no Excel
mapa_abas = {
  'df_clientes': 'Clientes',
  'df_contratos': 'Contratos',
  'df_acionamentos': 'Acionamentos',
}

try:
  dfs = {
    df_name: pl.read_excel(target, sheet_name=sheet_name)
    for df_name, sheet_name in mapa_abas.items()
  }

  for var_name, df in dfs.items():
    globals()[var_name] = df

  print('✅ Arquivos carregados com sucesso!')
except FileNotFoundError:
  print('❌ Erro: Por favor, verifique se o arquivo esta no seu drive.')


✅ Arquivos carregados com sucesso!


## Verificando os tipos de dados
Quero entender, qual o tamanho dos dados, quais e quantas colunas existem e quais seus tipos. A tipagem será importante para saber se posso unir os dataframes sem erros.

In [75]:
for nome, df in dfs.items():
  print('shape:', df.shape)

  print(f'=== {nome} ===')
  print_dataframe_schema(df)   # se sua função já usa df.schema, ok

shape: (1000, 4)
=== df_clientes ===
shape: (4, 2)
┌──────────────┬────────┐
│ column_name  ┆ dtype  │
│ ---          ┆ ---    │
│ str          ┆ str    │
╞══════════════╪════════╡
│ cliente_id   ┆ Int64  │
│ idade        ┆ Int64  │
│ cidade       ┆ String │
│ renda_mensal ┆ Int64  │
└──────────────┴────────┘
shape: (3000, 5)
=== df_contratos ===
shape: (5, 2)
┌──────────────────┬────────┐
│ column_name      ┆ dtype  │
│ ---              ┆ ---    │
│ str              ┆ str    │
╞══════════════════╪════════╡
│ contrato_id      ┆ Int64  │
│ cliente_id       ┆ Int64  │
│ valor_contrato   ┆ Int64  │
│ status_pagamento ┆ String │
│ data_inicio      ┆ Date   │
└──────────────────┴────────┘
shape: (10000, 4)
=== df_acionamentos ===
shape: (4, 2)
┌──────────────────┬────────┐
│ column_name      ┆ dtype  │
│ ---              ┆ ---    │
│ str              ┆ str    │
╞══════════════════╪════════╡
│ acionamento_id   ┆ Int64  │
│ contrato_id      ┆ Int64  │
│ tipo_contato     ┆ String │
│ data_acio

# Preparando os dados

In [76]:
# Normalizando, abreviando e padronizando os nomes de colunas. Nome de colunas não seguiam o mesmo padrão
# df_clientes
coluna_alvo = 'cliente_id'
if coluna_alvo in df_clientes.columns:
  rename = {'cliente_id':'id_cliente'}
  df_clientes = df_clientes.rename(rename)

# df_contratos
df_contratos = normalize_columns(df_contratos) # normalizando os nomes das colunas
coluna_alvo = 'data_inicio'
if coluna_alvo in df_contratos.columns:
  rename = {
    'contrato_id':'id_contrato',
    'cliente_id':'id_cliente',
    'data_inicio':'dt_contratacao',
    'valor_contrato':'vlr_contrato',
    'status_pagamento':'st_pagto'
  }
  df_contratos = df_contratos.rename(rename)

print(df_contratos.columns)

# df_acionamentos
df_acionamentos = normalize_columns(df_acionamentos)
coluna_alvo ='data_acionamento'
if coluna_alvo in df_acionamentos.columns:
  rename = {
    'acionamento_id':'id_acionamento',
    'contrato_id':'id_contrato',
    'data_acionamento':'dt_acionamento',
    'tipo_contato':'descr_canal'
    }
  df_acionamentos = df_acionamentos.rename(rename)
print(df_acionamentos.columns)

['id_contrato', 'id_cliente', 'vlr_contrato', 'st_pagto', 'dt_contratacao']
['id_acionamento', 'id_contrato', 'descr_canal', 'dt_acionamento']


## Adicionado colunas relevantes

### Faixa de renda [df_clientes]

In [77]:
df_clientes  = df_clientes.sort('renda_mensal')
# print_dataframe(df_consolidado.head(3))

df_faixas_renda = pl.DataFrame(
  {
    'renda_mensal': [0, 2000, 5000,7000, 10000, 20000],
    'faixa_renda': ['01.<2k', '02.2k A 5k', '03.5k A 7k','04.7k A 10k', '05.10k A 20k', '06.20k+'],
  }
)

coluna_alvo_renda = 'faixa_renda'

if coluna_alvo_renda not in df_clientes.columns:
  df_clientes = df_clientes.join_asof(
    df_faixas_renda,
    on='renda_mensal',
    strategy='backward'
  )

# clientes sem renda declarada => "07. Sem Info"
df_clientes = df_clientes.with_columns(
  pl.when(pl.col("renda_mensal").is_null()).then(pl.lit("07. Sem Info")).otherwise(pl.col("faixa_renda")).alias("faixa_renda")
)

print('Amostra dos dados:')
print_dataframe(df_clientes.head(3))

print('\nVerificação das faixas criadas:')
print(df_clientes.select(['renda_mensal', 'faixa_renda']).unique('faixa_renda').sort('renda_mensal'))
# print_dataframe(df_consolidado.select(['renda_mensal', 'faixa_renda']).unique().sort('renda_mensal'))

Amostra dos dados:
shape: (3, 5)
┌────────────┬───────┬────────────────┬──────────────┬─────────────┐
│ id_cliente ┆ idade ┆ cidade         ┆ renda_mensal ┆ faixa_renda │
│ ---        ┆ ---   ┆ ---            ┆ ---          ┆ ---         │
│ i64        ┆ i64   ┆ str            ┆ i64          ┆ str         │
╞════════════╪═══════╪════════════════╪══════════════╪═════════════╡
│ 795        ┆ 35    ┆ Curitiba       ┆ 1006         ┆ 01.<2k      │
│ 274        ┆ 50    ┆ Rio de Janeiro ┆ 1007         ┆ 01.<2k      │
│ 513        ┆ 46    ┆ Porto Alegre   ┆ 1040         ┆ 01.<2k      │
└────────────┴───────┴────────────────┴──────────────┴─────────────┘

Verificação das faixas criadas:
shape: (5, 2)
┌──────────────┬──────────────┐
│ renda_mensal ┆ faixa_renda  │
│ ---          ┆ ---          │
│ i64          ┆ str          │
╞══════════════╪══════════════╡
│ 1006         ┆ 01.<2k       │
│ 2019         ┆ 02.2k A 5k   │
│ 5000         ┆ 03.5k A 7k   │
│ 7031         ┆ 04.7k A 10k  │
│ 10028    

### Faixa etária [df_clientes]

In [78]:
df_clientes = df_clientes.sort('idade')
# print_dataframe(df_consolidado.head(3))

df_faixas_idade = pl.DataFrame(
  {
    'idade': [0,18, 26, 36, 46, 60],
    'faixa_etaria': ['01.<18','02.18 A 25', '03.26 A 35', '04.36 A 45', '05.46 A 60', '06.>60'],
  }
)

coluna_alvo = 'faixa_etaria'
if coluna_alvo not in df_clientes.columns:
  df_clientes = df_clientes.join_asof(
    df_faixas_idade,
    on='idade',
    strategy='backward'
  )

# clientes sem idade declarada => "07. Sem Info"
df_clientes = df_clientes.with_columns(
  pl.when(pl.col("idade").is_null()).then(pl.lit("07. Sem Info")).otherwise(pl.col("faixa_etaria")).alias("faixa_etaria")
)

print('Amostra dos dados:')
print_dataframe(df_clientes.head(3))

# verificado se as classes estão corretas
print_dataframe(df_clientes.select(['idade','faixa_etaria']).unique('faixa_etaria').sort('idade'))
# print_dataframe(df_consolidado.select(['idade','faixa_etaria']).unique().sort('idade')) #verifica para as demais idades, comentado pois gera uma lista grande

Amostra dos dados:
shape: (3, 6)
┌────────────┬───────┬──────────────┬──────────────┬─────────────┬──────────────┐
│ id_cliente ┆ idade ┆ cidade       ┆ renda_mensal ┆ faixa_renda ┆ faixa_etaria │
│ ---        ┆ ---   ┆ ---          ┆ ---          ┆ ---         ┆ ---          │
│ i64        ┆ i64   ┆ str          ┆ i64          ┆ str         ┆ str          │
╞════════════╪═══════╪══════════════╪══════════════╪═════════════╪══════════════╡
│ 568        ┆ 18    ┆ Porto Alegre ┆ 1405         ┆ 01.<2k      ┆ 02.18 A 25   │
│ 949        ┆ 18    ┆ São Paulo    ┆ 1557         ┆ 01.<2k      ┆ 02.18 A 25   │
│ 698        ┆ 18    ┆ Porto Alegre ┆ 1935         ┆ 01.<2k      ┆ 02.18 A 25   │
└────────────┴───────┴──────────────┴──────────────┴─────────────┴──────────────┘
shape: (5, 2)
┌───────┬──────────────┐
│ idade ┆ faixa_etaria │
│ ---   ┆ ---          │
│ i64   ┆ str          │
╞═══════╪══════════════╡
│ 18    ┆ 02.18 A 25   │
│ 26    ┆ 03.26 A 35   │
│ 36    ┆ 04.36 A 45   │
│ 46    ┆ 05.46

### Faixa de atraso

In [79]:
df_contratos = df_contratos.with_columns(
  pl.when(pl.col('st_pagto')=='Em dia').then(pl.lit('Em dia'))
  .when(pl.col('st_pagto')=='Inadimplente').then(pl.lit('0 A 30 dias')).otherwise(pl.lit('>30 dias')).alias('faixa_atraso')
)

### Dados do último acionamento [df_acionamentos]

In [80]:
#obtendo o ultimo acionamento
if 'dt_acionamento_right' not in df_acionamentos.columns:
  df_ultimo_acionamento = (df_acionamentos.sort('dt_acionamento', descending=True).select(['id_contrato','dt_acionamento','descr_canal']).unique(subset=['id_contrato'], keep='first'))
  df_acionamentos = df_acionamentos.join(df_ultimo_acionamento, on='id_contrato', how='left')

if 'dt_acionamento_right' in df_acionamentos.columns:
  rename = {
    'dt_acionamento_right':'dt_ult_acionamento',
    'descr_canal_right':'canal_ult_acionamento'
  }
  df_acionamentos = df_acionamentos.rename(rename)

print_dataframe(df_acionamentos.head(3))

shape: (3, 6)
┌────────────────┬─────────────┬─────────────┬────────────────┬────────────────────┬───────────────────────┐
│ id_acionamento ┆ id_contrato ┆ descr_canal ┆ dt_acionamento ┆ dt_ult_acionamento ┆ canal_ult_acionamento │
│ ---            ┆ ---         ┆ ---         ┆ ---            ┆ ---                ┆ ---                   │
│ i64            ┆ i64         ┆ str         ┆ date           ┆ date               ┆ str                   │
╞════════════════╪═════════════╪═════════════╪════════════════╪════════════════════╪═══════════════════════╡
│ 1              ┆ 2004        ┆ WhatsApp    ┆ 2023-06-24     ┆ 2024-02-20         ┆ WhatsApp              │
│ 2              ┆ 1983        ┆ Email       ┆ 2023-03-29     ┆ 2024-02-20         ┆ Email                 │
│ 3              ┆ 2229        ┆ WhatsApp    ┆ 2024-02-03     ┆ 2024-02-03         ┆ WhatsApp              │
└────────────────┴─────────────┴─────────────┴────────────────┴────────────────────┴───────────────────────┘


### Data do primeiro contrato (data de entrada na base)

In [93]:
contratos_pri_contracao = df_contratos.select(['id_cliente','dt_contratacao']).sort(['id_cliente','dt_contratacao'],descending=[False,False]).unique('id_cliente')
rename = {'dt_contratacao':'dt_pri_contracao'}
contratos_pri_contracao = contratos_pri_contracao.rename(rename)
if 'dt_pri_contracao' not in df_clientes.columns:
  df_clientes = df_clientes.join(contratos_pri_contracao, on='id_cliente', how='left')

print_dataframe(df_clientes.head(3))

cliente_sem_contratacao = df_clientes.filter(pl.col('dt_pri_contracao').is_null())
print(f'\n⚠ Atenção: Existem {cliente_sem_contratacao.select('id_cliente').count().item()} clientes sem nenhuma contração associada. Por favor, Verifique junto ao responsável do setor de cadastros.')

print_dataframe(cliente_sem_contratacao)

# gerado aqui, pois não esta sendo utilizado

# filename = 'clientes_sem_contratacao.xlsx'
# output = {'clientes_sem_contratacao': cliente_sem_contratacao}
# gerar_outputs_excel(output, filename, output_path)

shape: (3, 7)
┌────────────┬───────┬──────────────┬──────────────┬─────────────┬──────────────┬──────────────────┐
│ id_cliente ┆ idade ┆ cidade       ┆ renda_mensal ┆ faixa_renda ┆ faixa_etaria ┆ dt_pri_contracao │
│ ---        ┆ ---   ┆ ---          ┆ ---          ┆ ---         ┆ ---          ┆ ---              │
│ i64        ┆ i64   ┆ str          ┆ i64          ┆ str         ┆ str          ┆ date             │
╞════════════╪═══════╪══════════════╪══════════════╪═════════════╪══════════════╪══════════════════╡
│ 568        ┆ 18    ┆ Porto Alegre ┆ 1405         ┆ 01.<2k      ┆ 02.18 A 25   ┆ 2022-07-08       │
│ 949        ┆ 18    ┆ São Paulo    ┆ 1557         ┆ 01.<2k      ┆ 02.18 A 25   ┆ null             │
│ 698        ┆ 18    ┆ Porto Alegre ┆ 1935         ┆ 01.<2k      ┆ 02.18 A 25   ┆ 2022-08-18       │
└────────────┴───────┴──────────────┴──────────────┴─────────────┴──────────────┴──────────────────┘

⚠ Atenção: Existem 59 clientes sem nenhuma contração associada. Por favor, V

## Gerando arquivos star schema

In [90]:
path = input_path + 'star_schema/'
filename = 'clientes.xlsx'
output = {'clentes': df_clientes}
gerar_outputs_excel(output, filename, path)

filename = 'contratos.xlsx'
output = {'contratos': df_contratos}
gerar_outputs_excel(output, filename, path)

filename = 'acionamentos.xlsx'
output = {'acionamentos': df_acionamentos}
gerar_outputs_excel(output, filename, path)

✅ Excel gerado em: /content/drive/MyDrive/00. Cases/case_loft_202512/input/star_schema/clientes.xlsx
✅ Excel gerado em: /content/drive/MyDrive/00. Cases/case_loft_202512/input/star_schema/contratos.xlsx
✅ Excel gerado em: /content/drive/MyDrive/00. Cases/case_loft_202512/input/star_schema/acionamentos.xlsx


### Explorando os dados de cada dataframe

In [91]:
print_dataframe(df_clientes.describe())
print_dataframe(df_contratos.describe())
print_dataframe(df_acionamentos.describe())

# convertendo as colunas categoricas para categorias

shape: (9, 8)
┌────────────┬────────────┬──────────┬────────────────┬──────────────┬──────────────┬──────────────┬────────────────────────────┐
│ statistic  ┆ id_cliente ┆ idade    ┆ cidade         ┆ renda_mensal ┆ faixa_renda  ┆ faixa_etaria ┆ dt_pri_contracao           │
│ ---        ┆ ---        ┆ ---      ┆ ---            ┆ ---          ┆ ---          ┆ ---          ┆ ---                        │
│ str        ┆ f64        ┆ f64      ┆ str            ┆ f64          ┆ str          ┆ str          ┆ str                        │
╞════════════╪════════════╪══════════╪════════════════╪══════════════╪══════════════╪══════════════╪════════════════════════════╡
│ count      ┆ 1000.0     ┆ 1000.0   ┆ 1000           ┆ 1000.0       ┆ 1000         ┆ 1000         ┆ 941                        │
│ null_count ┆ 0.0        ┆ 0.0      ┆ 0              ┆ 0.0          ┆ 0            ┆ 0            ┆ 59                         │
│ mean       ┆ 500.5      ┆ 43.819   ┆ null           ┆ 8001.778     ┆ null 

In [ ]:
# Clientes
dfc = df_clientes.to_pandas()
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(dfc["idade"], bins=10, edgecolor="black")
plt.title("Histograma - Idade clientes")
plt.xlabel("Idade")
plt.ylabel("Frequência")

plt.subplot(1,2,2)
plt.hist(dfc["renda_mensal"], bins=10, edgecolor="black")
plt.title("Histograma - Renda mensal")
plt.xlabel("Renda mensal")
plt.ylabel("Frequência")
plt.tight_layout()
plt.savefig(output_path + 'imagens/hist_renda.png')
plt.show()

# Contratos
dfct = df_contratos.to_pandas()
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.hist(dfct["vlr_contrato"], bins=10, edgecolor="black")
plt.title("Histograma - Valor do contrato")
plt.xlabel("Valor contrato")
plt.ylabel("Frequência")

plt.subplot(1,2,2)
plt.hist(dfct["dt_contratacao"], bins=12, edgecolor="black")
plt.title("Histograma - Datas de contratação")
plt.xlabel("Data")
plt.ylabel("Frequência")
plt.tight_layout()
plt.savefig(output_path + 'imagens/hist_safra_contr.png')
plt.show()

# Acionamentos por contrato
qtd_acion_por_ctt = (df_acionamentos.group_by("id_contrato").agg(pl.len().alias("qtd_acionamentos")).to_pandas())

plt.figure(figsize=(6,4))
plt.hist(qtd_acion_por_ctt["qtd_acionamentos"], bins=10, edgecolor="black")
plt.title("Histograma - Qtde de acionamentos por contrato")
plt.xlabel("Qtde acionamentos")
plt.ylabel("Frequência")
plt.tight_layout()
plt.savefig(output_path + 'imagens/hist_qtdAcio_ctt.png')
plt.show()

### Gerando um dataframe consolidado

Escolhido devido ao volume de dados ser pequeno, para simplificar a exploração dos dados e descoberta de padrões;

Iniciaremos pela tabela de 'Contratos' pois um cliente pode ter multiplos contratos e um contrato pode ou não ter acionamentos.

In [ ]:
# Adicionando os dados dos clientes
df_consolidado = df_contratos.join(df_clientes, on='id_cliente', how='left')

# Adicionando os dados dos df_acionamentos
df_consolidado = df_consolidado.join(df_acionamentos, on='id_contrato', how='left')

# simplificando e adicionando contadores de distintos par aganhar velocidade nas analises
df_consolidado = (
  df_consolidado.sort('dt_acionamento').with_columns([ # classificando para garantir que o contador vai estar na maior dt_acionamento
    pl.when(pl.col('id_contrato').is_first_distinct().over('id_contrato')).then(1).otherwise(0).alias('flag_ctt'),
    pl.when(pl.col('id_cliente').is_first_distinct().over('id_cliente')).then(1).otherwise(0).alias('flag_cli'),
    pl.when(pl.col('dt_ult_acionamento').is_not_null()).then(1).otherwise(0).alias('flag_acionado'),
    pl.col('dt_contratacao').dt.strftime('%Y%m').cast(pl.Int64).alias('safra_contratacao'),
    pl.col('dt_acionamento').dt.strftime('%Y%m').cast(pl.Int64).alias('anomes_acionamento'),
    pl.col('dt_ult_acionamento').dt.strftime('%Y%m').cast(pl.Int64).alias('anomes_ult_acionamento'),

  ])
)

print('shape:', df_consolidado.shape)
print('columns:', df_consolidado.columns) # será util para selecionar ou reeordenar as colunas sem precisar digitar tudo
# print_dataframe(df_consolidado.sort('id_contrato').head(20)) # função proprietária para formatar a exibição do dataframe
print_dataframe_schema(df_consolidado) # função proprietária para formatar a exibição do schema
verificar_nulos(df_consolidado) # função proprietária para verificação de nulos.
print('\n')

# Gerando resumos

### Distribuição dos clientes por status

In [ ]:
# Define prioridade de status
status_prioridade = {
  'Inadimplente': 3,
  'Atrasado': 2,
  'Em dia': 1,
}

# Base nível cliente × safra × status, mantendo 1 status por cliente/safra
base_cli_safra = (
  df_consolidado.select(['id_cliente', 'safra_contratacao', 'st_pagto'])
  .with_columns(pl.col('st_pagto').replace(status_prioridade).alias('prio_st')).sort(['id_cliente', 'safra_contratacao', 'prio_st'],descending=[False, False, True])
  .unique(subset=['id_cliente', 'safra_contratacao'], keep='first').select(['id_cliente', 'safra_contratacao', 'st_pagto'])
)

# Qtde de clientes por safra × status
agg_safra_status = base_cli_safra.group_by(['safra_contratacao', 'st_pagto']).agg(pl.n_unique('id_cliente').alias('qtd_clientes'))

# Total de clientes por safra
agg_tot_safra = base_cli_safra.group_by('safra_contratacao').agg(pl.n_unique('id_cliente').alias('qtd_cli_safra'))

# % de clientes por status dentro de cada safra (sem arredondar aqui)
agg_pct = (
  agg_safra_status.join(agg_tot_safra, on='safra_contratacao', how='left').with_columns((pl.col('qtd_clientes') / pl.col('qtd_cli_safra') * 100).alias('pct_cli'))
)

# print(agg_pct)

# Pivot safra × status
heat_df = agg_pct.select(['safra_contratacao', 'st_pagto', 'pct_cli']).pivot(index='safra_contratacao', columns='st_pagto', values='pct_cli').sort('safra_contratacao')

# Converte para pandas, define índice e ordena colunas
heat_pd = heat_df.to_pandas().set_index('safra_contratacao')
ordem_cols = ['Em dia', 'Atrasado', 'Inadimplente']
heat_pd = heat_pd[ordem_cols]

heat_pd["Inadimplencia total"] = heat_pd["Atrasado"] + heat_pd["Inadimplente"]

# Confere se cada linha fecha ~100%
# print('Soma por safra (sem arredondar):')
# print(heat_pd.sum(axis=1))

# Normaliza cada coluna (0–1) apenas para cor
norm_values = (heat_pd - heat_pd.min()) / (heat_pd.max() - heat_pd.min())

# Plota heatmap: cor usa norm_values, labels usam % reais arredondados
plt.figure(figsize=(9, 6))
sns.heatmap(
  norm_values,
  annot=heat_pd.round(1),
  fmt='.1f',
  cmap='YlOrRd',
  cbar=False,
  linewidths=0.5,
  linecolor='white'
)
plt.title('% de clientes por status de pagamento × safra de contratação')
plt.xlabel('Status de pagamento')
plt.ylabel('Safra de contratação (YYYYMM)')
plt.yticks(rotation=0)
plt.savefig(output_path + 'imagens/pctCli_SafraContr_stPagto.png')
plt.show()

#### Análise auxiliar do mapa de calor acima

In [ ]:
add_inad_geral =agg_pct.filter(pl.col("st_pagto").is_in(["Atrasado", "Inadimplente"])).group_by("safra_contratacao").agg(
  pl.lit("Inadimplencia total").alias("st_pagto"),
  pl.col("qtd_clientes").sum().alias("qtd_clientes"),
  pl.col("qtd_cli_safra").max().alias("qtd_cli_safra"),  # mesmo denominador
  pl.col("pct_cli").sum().alias("pct_cli")
)

auxiliar = pl.concat([agg_pct, add_inad_geral], how="vertical")

# print_dataframe(agg_pct_ext)

status = 'Atrasado'
filtro = round(auxiliar.filter(pl.col('st_pagto')==status).select(['pct_cli']).mean().item(),2)
print(f'O percentual médio de clientes no status {status} é de :' ,filtro,'\n')

filtro = round(auxiliar.filter((pl.col('st_pagto')==status)).select(['pct_cli']).count().item(),2)
print(f'A quantidade de clientes no status {status} é de :' ,filtro,'\n')

filtro = round(auxiliar.filter((pl.col('st_pagto')==status) & (pl.col('pct_cli')>=60)).select(['pct_cli']).count().item(),2)
print(f'A quantidade de clientes no status {status} e >60% é de :' ,filtro,'\n')

## Distribuição dos clientes adimplentes por faixa_etaria e canal

In [ ]:
# Base por cliente, faixa etária, último canal e status
base_canal = (
  df_consolidado.select(['id_cliente','faixa_etaria','canal_ult_acionamento','st_pagto']).unique(['id_cliente','canal_ult_acionamento'])  # evita duplicar cliente/canal
)

# Taxa de contratos Em dia por faixa_etaria × canal
taxa_canal = (
  base_canal.group_by(['faixa_etaria','canal_ult_acionamento']).agg([
    pl.len().alias('qtd_clientes'),
    (pl.col('st_pagto') == 'Em dia').cast(pl.Int32).sum().alias('qtd_em_dia'),
  ]).with_columns((pl.col('qtd_em_dia') / pl.col('qtd_clientes') * 100).round(2).alias('pct_em_dia'))
)

# print('Tabela resumo por faixa_etaria × canal:')
# print_dataframe(taxa_canal)

# Pivot em Polars para formar a matriz do heatmap
tabela_pl = (
  taxa_canal
  .pivot(
    values='pct_em_dia',
    index='faixa_etaria',
    on='canal_ult_acionamento'
  )
  .sort('faixa_etaria')
)

# print('Matriz de % Em dia (linhas = faixa_etaria, colunas = canal):')
# print_dataframe(tabela_pl)

# Heatmap em Seaborn
tabela = tabela_pl.to_pandas().set_index('faixa_etaria')
plt.figure(figsize=(8,6))
sns.heatmap(tabela,annot=True,fmt='.1f',cmap='Greens')
plt.title('% de clientes Em dia por faixa etária × canal')
plt.xlabel('Canal do último acionamento')
plt.ylabel('Faixa etária')
plt.tight_layout()
plt.savefig(output_path + 'imagens/canal_fxEtaria_pctClientes.png')
plt.show()

#### Análise auxiliar do mapa de calor acima

## Mapenado o perfil dos clientes inadimplentes

### Inadimplência por safra de contratação

In [ ]:
taxa_inad_vol = df_consolidado.unique('id_cliente')

taxa_inad_vol = (
  taxa_inad_vol.filter(pl.col('flag_ctt')==1).group_by("safra_contratacao").agg([
    pl.len().alias("qtd_total"),
    ((pl.col("st_pagto") == 'Inadimplente')).sum().alias("qtd_inad"),
    ((pl.col("st_pagto") == "Atrasado")).sum().alias("qtd_atrasado")
  ])
  .with_columns(
    (pl.col("qtd_inad") / pl.col("qtd_total") * 100).round(2).alias("pct_inadimplencia"),
    (pl.col("qtd_atrasado") / pl.col("qtd_total") * 100).round(2).alias("pct_atrasado"),
  ).sort("safra_contratacao")
)
taxa_inad_vol = taxa_inad_vol.with_columns((pl.col('pct_inadimplencia') + pl.col('pct_atrasado')).alias('pct_inad_total'))
taxa_inad_vol = taxa_inad_vol.select(['safra_contratacao','pct_inadimplencia','pct_atrasado','pct_inad_total'])

print_dataframe(taxa_inad_vol)
# print_dataframe(taxa_inad_vol.filter(pl.col('pct_inad_total')>45))


print(f'Inadimplencia total: {round(taxa_inad_vol.select('pct_inad_total').mean().item(),2)}')
print(f'Inadimplencia total < 45%: {round(taxa_inad_vol.filter(pl.col('pct_inad_total')<45).select('pct_inad_total').mean().item(),2)}')
print(f'Inadimplencia total > 45%: {round(taxa_inad_vol.filter(pl.col('pct_inad_total')>45).select('pct_inad_total').mean().item(),2)}')

### Inadimplencia por faixa etária e renda

In [ ]:
# base de clientes únicos com faixa_etaria, faixa_renda e status pagamento
clientes = df_consolidado.select(["id_cliente","st_pagto","faixa_etaria","faixa_renda"]).unique("id_cliente")

# Garante que todas as faixas etarárias e de renda sejam exibidas
faixas_idade = df_faixas_idade.select('faixa_etaria').sort('faixa_etaria').unique()
faixas_renda = df_faixas_renda.select("faixa_renda").sort('faixa_renda').unique()

# Gerao produto cartesiano de faixas (todas as combinações possíveis)
grid = faixas_renda.join(faixas_idade, how="cross")

# Junta o grid com os dados calculados (taxa_inad)
taxa_inad_cli =grid.join(clientes,on=["faixa_etaria","faixa_renda"],how="left")
taxa_inad_cli = taxa_inad_cli.group_by(["faixa_etaria","faixa_renda"]).agg(
  pl.len().alias("qtd_clientes_total"),
  (pl.col("st_pagto") == "Inadimplente").cast(pl.Int32).sum().alias("qtd_inad")
).with_columns((pl.col("qtd_inad") / pl.col("qtd_clientes_total") * 100).round(2).alias("pct_inad")).with_columns(pl.col("pct_inad").fill_null(0))


taxa_max = 20 # definindo a taxa base para verificar
print('Ofensores em taxa de inadimplência')
print(taxa_inad_cli.filter(pl.col('pct_inad')>=taxa_max))

### Gerando um mapa de calor para facilitar a identificação do perfil dos clientes mais críticos

In [ ]:
# Faz o pivot em cima da grade completa
tabela_pl = (
  taxa_inad_cli.pivot(
    values="pct_inad",
    index="faixa_renda",
    columns="faixa_etaria"
  )
).sort("faixa_renda")

# print_dataframe(tabela_pl)


# ordem desejada das faixas etárias
ordem_faixa_etaria = [
  "01.<18",
  "02.18 A 25",
  "03.26 A 35",
  "04.36 A 45",
  "05.46 A 60",
  "06.>60",
]

tabela = tabela_pl.to_pandas().set_index("faixa_renda")

# garante que as colunas apareçam na ordem definida
tabela = tabela[ordem_faixa_etaria]

plt.figure(figsize=(8,6))
sns.heatmap(tabela, annot=True, fmt=".1f", cmap="Reds")
plt.title("Taxa de inadimplência por faixa etária × faixa de renda")
plt.xlabel("Faixa etária")
plt.ylabel("Faixa de renda")
plt.tight_layout()
plt.savefig(output_path + 'imagens/txInad_fxEtaria_fxRenda.png')
plt.show()

## Verificando contratos orfãos na tabela de acionamentos

In [ ]:
# contratos presentes em df_contratos
contratos_unicos = df_contratos.select("id_contrato").unique()

# acionamentos sem contrato na base de contratos
acionamentos_orfaos = df_acionamentos.join(contratos_unicos, on="id_contrato", how="anti")

print("Acionamentos sem contrato correspondente em df_contratos:")
print_dataframe(acionamentos_orfaos.head(20))
print("Total órfãos:", acionamentos_orfaos.height)

#gerando arquivo excel caso aparecem acionamentos sem um contrato na tabela contratos
if acionamentos_orfaos.height > 0:
  filename = 'acionamentos_orfaos.xlsx'
  output = {'orfãos': acionamentos_orfaos}
  gerar_outputs_excel(output, output_path, filename)

## Calculando a eficácia

In [ ]:
ctt_em_dia = df_consolidado.filter((pl.col('st_pagto') == 'Em dia') & (pl.col('flag_ctt')==1))
ctt_totais = df_consolidado.filter((pl.col('flag_ctt')==1))
# meta_eficacia = 0.7

eficacia = round((ctt_em_dia.shape[0] / ctt_totais.shape[0]) * 100,2)
print(
  f'Foi identificado que tivemos {eficacia}% de eficácia.'
  f'Sendo que resolvemos {ctt_em_dia.shape[0]} de {ctt_totais.shape[0]} contrato até o momento'
)

## Tentativas por status de pagamento

In [ ]:
total_contratos = df_consolidado.select(pl.sum('flag_ctt').alias('total_contratos')).item()

resumo_tentativas = (
  df_consolidado.group_by('st_pagto').agg([
    pl.len().alias('qtd_acionamentos'), #contar todas as linhas reflete a quantidade de acionamentos
    pl.sum('flag_ctt').alias('qtd_contratos') #gera a quantidade de contratos por status de pagamento
  ])
  .with_columns(
    (pl.col('qtd_acionamentos') / pl.col('qtd_contratos')).alias('media_tentativas'),
    (pl.col('qtd_contratos') / total_contratos * 100).alias('pct_distr'),
  )
  .with_columns([
    pl.col('media_tentativas').round(2),
    pl.col('pct_distr').round(2),
  ]).sort('st_pagto')
)

print_dataframe(resumo_tentativas)
print('O total de contrato é de',resumo_tentativas.select('qtd_contratos').sum().item())

## Teste de hipóteses

Pergunta que motivou o teste:
A diferença na média de acionamentos entre os st_pagto é relevantes?

Motivo da escolha:
Como o problema é comparar a média de uma variável numérica entre três grupos independentes. Esse cenário é o caso clássico de ANOVA de uma via: 1 fator categórico com 3+ níveis, 1 variável numérica e grupos independentes.
<br><br>

Hipótese nula (H₀): não existe diferença entre as médias de quantidade de acionamentos por contrato entre os diferentes status de pagamento (Em dia, Atrasado e Inadimplente).

Hipótese alternativa (H₁): existe pelo menos uma diferença entre as médias de quantidade de acionamentos por contrato em pelo menos um dos status de pagamento.

Interpretação:

Se p_value > 0.05 (ou o nível que você definir), não há evidência estatística de diferença nas médias de tentativas entre os 3 status.

Se p_value ≤ 0.05, existe diferença significativa e você pode depois fazer testes pós-hoc (Tukey, por exemplo) para ver quais pares diferem.

In [ ]:
# Quantidade de acionamentos por contrato + status final
qtd_por_contrato = df_consolidado.group_by(['id_contrato', 'st_pagto']).agg(pl.len().alias('qtd_acionamentos'))

# Converte para pandas para usar scipy
qtd_pd = qtd_por_contrato.to_pandas()

# Separa os grupos
g_em_dia = qtd_pd.loc[qtd_pd['st_pagto'] == 'Em dia', 'qtd_acionamentos']
g_atrasado = qtd_pd.loc[qtd_pd['st_pagto'] == 'Atrasado', 'qtd_acionamentos']
g_inad = qtd_pd.loc[qtd_pd['st_pagto'] == 'Inadimplente', 'qtd_acionamentos']

# ANOVA de uma via
f_stat, p_value = stats.f_oneway(g_em_dia, g_atrasado, g_inad)

alpha = 0.05  # nível de significância

print(f'Resultado ANOVA - F = {f_stat:.4f}, p-valor = {p_value:.6f}')

if p_value > alpha:
  print(
    'Veredito: Não há evidência estatisticamente significativa, ao nível de 5%, '
    'de que a variação da quantidade média de tentativas seja relevante entre os status de pagamento '
    '(Em dia, Atrasado, Inadimplente).'
  )
else:
  print(
    'Veredito: Há evidência estatisticamente significativa, ao nível de 5%, '
    'de que ao menos um status (Em dia, Atrasado, Inadimplente) possui '
    'quantidade média de tentativas diferente dos demais. '
    'Recomenda-se aplicar testes pós-hoc (ex.: Tukey) para identificar quais pares diferem.'
  )


## Analisando outliers de acionamentos

A piorização foi definida de acordo com a regra de negócio:

- 'Inadimplentes' são clientes mais complicados de recuperar e exigem muito mais esforço de cobrança;
- 'Atrasados' são clientes que tiveram percausos e acabaram não conseguindo pagar a dívida momentâneamente, mas costumavam pagar com recorrência;
- 'Em Dia' são cliente adimplentes;



### Criando recomendações para os casos não acionados

In [ ]:
# não acionados em aberto, não faz sentido acionar contrato 'Em dia'
filtro = df_consolidado.filter((pl.col('flag_acionado')==0) & (pl.col('st_pagto') != 'Em dia'))

nao_acionados_df = filtro.select([pl.col('id_contrato'),pl.col('st_pagto'),])
nao_acionados_df = nao_acionados_df.with_columns([
  pl.lit(0).cast(pl.UInt32).alias('qtd_acionamentos'),
  pl.lit('Avaliar (tecnologias/fluxos/parceiros)').alias('acao_gestor'),
  pl.lit(1).cast(pl.Int32).alias('pri'),
  pl.lit('não').alias('acionado_st'),
])

print_dataframe(nao_acionados_df.head())
print(nao_acionados_df.shape)

### Criando recomendações para os casos muito acionados


In [ ]:
# calcular limiar de outlier (ex.: acima do percentil 95)
filtro = df_consolidado.group_by(['id_contrato', 'st_pagto']).agg(pl.len().alias('qtd_acionamentos'))
limite_alto = (filtro.select(pl.col('qtd_acionamentos').quantile(0.95).alias('p95')).item())

# contratos com acionamento muito acima dos demais, então o esforço foi maior
outliers_acionamentos = (
  qtd_por_contrato.filter(pl.col('qtd_acionamentos') > limite_alto)
  .with_columns([
    # ação para o gestor
    pl.when(pl.col('st_pagto') == 'Em dia')
    .then(pl.lit('Fazer preventivo nos contratos já revertidos'))
    .when(pl.col('st_pagto') == 'Inadimplente')
    .then(pl.lit('Verificar aderência ao canal e efetividade dos contatos'))
    .when(pl.col('st_pagto') == 'Atrasado')
    .then(pl.lit('Verificar a efetividade e qualidadde dos contatos'))
    .otherwise(pl.lit('Revisar caso'))
    .alias('acao_gestor'),

    # prioridade: 2 = Inadimplente [0-30], 3 = Atrasado [>30], 4 = Em dia
    pl.when(pl.col('st_pagto') == 'Inadimplente').then(2)
    .when(pl.col('st_pagto') == 'Atrasado')
    .then(3).when(pl.col('st_pagto') == 'Em dia')
    .then(4).otherwise(4) # como estão em dia, só precisamos verificar ações preventivas para evitar novos atrasos
    .alias('pri'),
    pl.lit('sim').alias('acionado_st'),
  ])
)

### Unificando as recomendações

In [ ]:
val = (outliers_acionamentos.filter(pl.col('acionado_st') == 'não').height > 0)

if not val and not nao_acionados_df.is_empty():
  outliers_acionamentos = pl.concat([outliers_acionamentos, nao_acionados_df],how='vertical_relaxed')
  print(f'Adicionados {nao_acionados_df.height} contratos não acionados.')

outliers_acionamentos = outliers_acionamentos.sort(['pri','st_pagto','qtd_acionamentos'],descending=[False,True,True])
print_dataframe(outliers_acionamentos.head(5))

resumo_acao = outliers_acionamentos.select(['acao_gestor'])

## Analisando as distribuiuções 2 a 2

In [ ]:
# clientes únicos com perfil
clientes_unique = (
  df_consolidado.select(['st_pagto','id_cliente','idade','cidade','renda_mensal','faixa_renda','faixa_etaria'])
  .unique('id_cliente')
)

# contratos por grupo diretamente na base consolidada
# (ajuste o nome de id_contrato se estiver diferente)
contratos_grp = (
  df_consolidado.group_by(['st_pagto','cidade','faixa_renda','faixa_etaria']).agg(pl.n_unique('id_contrato')
  .alias('qtd_contratos'))
)

# ===== faixa_etaria =====
perfil_cli_faixa_etaria = clientes_unique.group_by(['st_pagto','faixa_etaria']).agg(pl.len().alias('qtd_clientes'))

perfil_ctt_faixa_etaria = (
  contratos_grp.group_by(['st_pagto','faixa_etaria']).agg(pl.col('qtd_contratos').sum().alias('qtd_contratos'))
)

perfil_faixa_etaria = (
  perfil_cli_faixa_etaria
  .join(perfil_ctt_faixa_etaria, on=['st_pagto','faixa_etaria'], how='left')
  .with_columns([
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum().over('st_pagto')).alias('pct_st_pagto'),
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum().over('faixa_etaria')).alias('pct_grupo'),
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum()).alias('pct_clientes'),
  ]).with_columns([
    (pl.col('pct_st_pagto') * 100).round(2),
    (pl.col('pct_grupo') * 100).round(2),
    (pl.col('pct_clientes') * 100).round(2),
  ])
  .rename({'faixa_etaria': 'grupo'})
  .with_columns(pl.lit('faixa_etaria').alias('analise_por'))
)

# ===== faixa_renda =====
perfil_cli_faixa_renda = (
  clientes_unique.group_by(['st_pagto','faixa_renda'])
  .agg(pl.len().alias('qtd_clientes'))
)

perfil_ctt_faixa_renda = (
  contratos_grp
  .group_by(['st_pagto','faixa_renda'])
  .agg(pl.col('qtd_contratos').sum().alias('qtd_contratos'))
)

perfil_faixa_renda = (
  perfil_cli_faixa_renda
  .join(perfil_ctt_faixa_renda, on=['st_pagto','faixa_renda'], how='left')
  .with_columns([
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum().over('st_pagto')).alias('pct_st_pagto'),
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum().over('faixa_renda')).alias('pct_grupo'),
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum()).alias('pct_clientes'),
  ]).with_columns([
    (pl.col('pct_st_pagto') * 100).round(2),
    (pl.col('pct_grupo') * 100).round(2),
    (pl.col('pct_clientes') * 100).round(2),
  ])
  .rename({'faixa_renda': 'grupo'})
  .with_columns(pl.lit('faixa_renda').alias('analise_por'))
)

# ===== cidade =====
perfil_cli_cidade = (
  clientes_unique.group_by(['st_pagto','cidade'])
  .agg(pl.len().alias('qtd_clientes'))
)

perfil_ctt_cidade = (
  contratos_grp
  .group_by(['st_pagto','cidade'])
  .agg(pl.col('qtd_contratos').sum().alias('qtd_contratos'))
)

perfil_cidade = (
  perfil_cli_cidade
  .join(perfil_ctt_cidade, on=['st_pagto','cidade'], how='left')
  .with_columns([
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum().over('st_pagto')).alias('pct_st_pagto'),
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum().over('cidade')).alias('pct_grupo'),
    (pl.col('qtd_clientes') / pl.col('qtd_clientes').sum()).alias('pct_clientes'),
  ]).with_columns([
    (pl.col('pct_st_pagto') * 100).round(2),
    (pl.col('pct_grupo') * 100).round(2),
    (pl.col('pct_clientes') * 100).round(2),
  ])
  .rename({'cidade': 'grupo'})
  .with_columns(pl.lit('cidade').alias('analise_por'))
)

# ===== unificar =====
perfil_unificado = pl.concat(
  [perfil_faixa_etaria, perfil_faixa_renda, perfil_cidade],
  how='vertical_relaxed'
)

colunas = ['st_pagto','analise_por','grupo','qtd_clientes','qtd_contratos','pct_st_pagto','pct_grupo','pct_clientes']

perfil_unificado = perfil_unificado.select(colunas).sort(['st_pagto','analise_por','grupo'])

print_dataframe(perfil_unificado)

# Gerando outputs

## Base consolidada para estudos pontuais

In [ ]:
# para o caso de bases pequena e interessante dar a oportunidade do gestor poder 'crincar' com o dados para conhecer melhor a sua base
filename = 'df_consolidado.xlsx'
output = {'df_consolidado':df_consolidado}
gerar_outputs_excel(output,filename,output_path)

## Recomendações de ações para outliers de acionamentos

In [ ]:
filename = 'outliers_acionamentos.xlsx'
output = {
  'outliers_acionamentos':outliers_acionamentos,
  # 'nao_acionados_df':nao_acionados_df
}
gerar_outputs_excel(output,filename,output_path)

## Análises das distribuições

In [ ]:
filename = 'analise_distribuicoes.xlsx'
output = {'geral':perfil_unificado}
gerar_outputs_excel(output,filename,'./')